# OPER 685 
# Data Wrangling
In this tutorial we are going to see some examples of data wrangling applied to first the Jeopardy database, and then the Titanic Dataset. The Jeopardy database has 250,000+ Jeopardy questions and answers along with several other data fields.
The Titanic Dataset is a dataset of passanger information from passangers onboard the Titatnic when it sank. 

Jeopardy Database tutorial originally from: https://www.nobledesktop.com/learn/python/data-wrangling-python-guide </p>
Titanic Database tutorial originally from: https://www.kaggle.com/jamesleslie/titanic-eda-wrangling-imputation

## 0. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1.0 Jeopardy Data

In [2]:
jeopardy = pd.read_csv("JEOPARDY_CSV.csv")

In [3]:
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [4]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

### 1.1 Missing Data

In [5]:
jeopardy.shape

(216930, 7)

In [6]:
jeopardy.isnull().sum()

Show Number    0
 Air Date      0
 Round         0
 Category      0
 Value         0
 Question      0
 Answer        2
dtype: int64

There are two missing answers.  Lets checkout those instances and see what we can do.

In [7]:
jeopardy[jeopardy[' Answer'].isnull()]

,Show Number,Air Date,Round,Category,Value,Question,Answer
94817,4346,2003-06-23,Jeopardy!,"GOING ""N""SANE",$200,"It often precedes ""and void""",NaN
143297,6177,2011-06-21,Double Jeopardy!,NOTHING,$400,"This word for ""nothing"" precedes ""and void"" to...",NaN


In this case since there is only 2 rows with missing data we can just 'mask' them (i.e., delete them).

In [8]:
jeopardy.drop(jeopardy.index[[94817,143297]], inplace = True)

In [9]:
jeopardy.shape

(216928, 7)

In [10]:
jeopardy.isnull().sum()

Show Number    0
 Air Date      0
 Round         0
 Category      0
 Value         0
 Question      0
 Answer        0
dtype: int64

### 1.2 Exploratory Data Analysis
Lets check the 10 most common questions and the 10 most common answers.  We'd expect some overlap in Answers being used more then once, but we'd expect none of the questions to be reused.

In [11]:
top_10A = jeopardy[' Answer'].value_counts()[:10]

In [12]:
top_10A

China         216
Australia     215
Japan         196
Chicago       194
France        193
India         185
California    180
Canada        176
Spain         171
Mexico        164
Name:  Answer, dtype: int64

We sorted all the answers by thier count and then grabbed first 10 rows

In [13]:
top_10Q = jeopardy[' Question'].value_counts()[:10]

In [14]:
top_10Q

[audio clue]                                     17
[video clue]                                     14
[filler]                                          5
(audio clue)                                      5
Abigail Smith                                     4
Hainan                                            4
Greenland                                         4
Dick Cheney                                       3
Argentina                                         3
It's the square root of the square root of 81     3
Name:  Question, dtype: int64

This is interesting 17+5 of the questions were just "audio clues", and 14 were "video clues".  Since the intent of this data wrangling is to clean the data to enable classification and these instances do not contain useful data we will remove them from our dataset.

In [15]:
# This gives me a boolean True/False for every instance
jeopardy[' Question'] == ("[audio clue]")

0         False
1         False
2         False
3         False
4         False
          ...  
216925    False
216926    False
216927    False
216928    False
216929    False
Name:  Question, Length: 216928, dtype: bool

In [16]:
#I then use that boolean array within the dataframe to select only the instances which are true
mask1 = jeopardy[jeopardy[' Question'] == ("[audio clue]")]
mask2 = jeopardy[jeopardy[' Question'] == ("[video clue]")]
mask3 = jeopardy[jeopardy[' Question'] == ("[filler]")]
mask4 = jeopardy[jeopardy[' Question'] == ("(audio clue)")]

In [17]:
#I now use those indices to drop the rows from the dataframe
jeopardy.drop(mask1.index, inplace = True)
jeopardy.drop(mask2.index, inplace = True)
jeopardy.drop(mask3.index, inplace = True)
jeopardy.drop(mask4.index, inplace = True)

In [18]:
#Double check to ensure those instances are gone
top_10Q = jeopardy[' Question'].value_counts()[:10]
top_10Q

Abigail Smith         4
Greenland             4
Hainan                4
Argentina             3
California            3
"Charlie Hustle"      3
Melanie Griffith      3
"You Must Love Me"    3
Dick Cheney           3
"I Hope I Get It"     3
Name:  Question, dtype: int64

Next we could go into Natural Language Processesing but that is beyond the scope of this course.

## 2.0 Titanic Data

In [30]:
titanic = pd.read_csv("titanic.csv")

In [31]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

### 2.1 Explore Data

In [33]:
titanic.shape

(1309, 12)

In [34]:
titanic.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,1309.0,NaN,NaN,NaN,655.0,378.020061,1.0,328.0,655.0,982.0,1309.0
Survived,891.0,NaN,NaN,NaN,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,1309.0,NaN,NaN,NaN,2.294882,0.837836,1.0,2.0,3.0,3.0,3.0
Name,1309,1307,"Kelly, Mr. James",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,1309,2,male,843,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1046.0,NaN,NaN,NaN,29.881138,14.413493,0.17,21.0,28.0,39.0,80.0
SibSp,1309.0,NaN,NaN,NaN,0.498854,1.041658,0.0,0.0,0.0,1.0,8.0
Parch,1309.0,NaN,NaN,NaN,0.385027,0.86556,0.0,0.0,0.0,0.0,9.0
Ticket,1309,929,CA. 2343,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,1308.0,NaN,NaN,NaN,33.295479,51.758668,0.0,7.8958,14.4542,31.275,512.3292


In [35]:
titanic['Survived'].value_counts()

0.0    549
1.0    342
Name: Survived, dtype: int64

### 2.2 Data Imputation 
There are a few columns with missing values. The Cabin column is missing over 1000 values, so we won't use that for predictions, but the Age columns is complete enough that we can fill in the missing values through imputation.

A simple option for the missing age values is to use the median age value. Let's go a little further and use each passenger's Title to estimate their age. E.g. if a passenger has the title of Dr, I will give them the median age value for all other passengers with the same title. We can use a regular expression to extract the title from the Name column. We will do this by finding the adjacent letters that are immediately followed by a full stop.

In [36]:
# create new Title column
titanic['Title'] = titanic['Name'].str.extract('([A-Za-z]+)\.', expand=True)

In [37]:
titanic['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Ms            2
Major         2
Mlle          2
Sir           1
Don           1
Countess      1
Jonkheer      1
Mme           1
Dona          1
Lady          1
Capt          1
Name: Title, dtype: int64

We will use the six most common titles, replacing all other titles with the most appropriate of these six

In [38]:
# replace rare titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',
           'Don': 'Mr', 'Mme': 'Mrs', 'Jonkheer': 'Mr', 'Lady': 'Mrs',
           'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
titanic.replace({'Title': mapping}, inplace=True)

In [39]:
# impute missing Age values using median of Title groups
title_ages = dict(titanic.groupby('Title')['Age'].median())

# create a column called "age_med" containing the average ages
# .apply applies the function  (lambda x: title_ages[x]) to column Title
# lambda x: title_ages[x] is a "lamda function", a quick function that doesn't need a formal def 
# in this case it takes what ever data is in that row in the title column, 
# looks it up in the dictonary title_ages and returns associated median age
titanic['age_med'] = titanic['Title'].apply(lambda x: title_ages[x])

# replace all missing ages (where age = na) with the value in this column
titanic['Age'].fillna(titanic['age_med'], inplace=True, )
del titanic['age_med']

In [40]:
titanic.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,1309.0,NaN,NaN,NaN,655.0,378.020061,1.0,328.0,655.0,982.0,1309.0
Survived,891.0,NaN,NaN,NaN,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,1309.0,NaN,NaN,NaN,2.294882,0.837836,1.0,2.0,3.0,3.0,3.0
Name,1309,1307,"Kelly, Mr. James",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,1309,2,male,843,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1309.0,NaN,NaN,NaN,29.562391,13.16536,0.17,22.0,30.0,35.5,80.0
SibSp,1309.0,NaN,NaN,NaN,0.498854,1.041658,0.0,0.0,0.0,1.0,8.0
Parch,1309.0,NaN,NaN,NaN,0.385027,0.86556,0.0,0.0,0.0,0.0,9.0
Ticket,1309,929,CA. 2343,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,1308.0,NaN,NaN,NaN,33.295479,51.758668,0.0,7.8958,14.4542,31.275,512.3292
